# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd

df = pd.read_csv('tic-tac-toe.csv')
df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [9]:
# Identify columns
target_col = next((c for c in df.columns if c.lower() == "class"), df.columns[-1])
grid_cols  = [c for c in df.columns if c != target_col]

# Make everything lowercase first
marks = df[grid_cols].apply(lambda s: s.astype(str).str.strip().str.lower())

# One-hot with fixed category order per square (x, o, b)
X_list = []
for col in grid_cols:
    cat = pd.Categorical(marks[col], categories=["x", "o", "b"])
    dummies = pd.get_dummies(cat, prefix=col)
    X_list.append(dummies.astype(int))
X = pd.concat(X_list, axis=1)

# y: map to 1 if X wins, else 0 (handles common label styles)
y = df[target_col].map({True:1, False:0}).astype(int)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [23]:
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense

# Input layer
model.add(Input(shape=(X_train.shape[1],)))

# Hidden layers (ReLU)
model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(16, activation='relu'))

# Output layer (2 units for classes 0 and 1)
model.add(Dense(2, activation='softmax'))

In [24]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [25]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,  # 20% of TRAIN used for validation
    shuffle=True,
    verbose=1
)

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6176 - loss: 0.6576 - val_accuracy: 0.7143 - val_loss: 0.5740
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6846 - loss: 0.6048 - val_accuracy: 0.7662 - val_loss: 0.5553
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7467 - loss: 0.5723 - val_accuracy: 0.7792 - val_loss: 0.5420
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7680 - loss: 0.5445 - val_accuracy: 0.7727 - val_loss: 0.5264
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7663 - loss: 0.5196 - val_accuracy: 0.7727 - val_loss: 0.5177
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7712 - loss: 0.4992 - val_accuracy: 0.7727 - val_loss: 0.5089
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7859 - loss: 0.4790 - val_accuracy: 0.7662 - val_loss: 0.4982
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7876 - loss: 0.4614 - val_accuracy: 0.7727 - val_loss

In [26]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f"\nTest accuracy: {test_acc:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9844 - loss: 0.0844 

Test accuracy: 0.9844


In [28]:
model.save("tic-tac-toe.keras")

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [29]:
from tensorflow.keras.models import load_model

model = load_model("tic-tac-toe.keras")

In [31]:
import numpy as np
np.random.seed(0)  # optional, for reproducibility

# Pick 5 random row positions
idx = np.random.choice(X_test.shape[0], 5, replace=False)

# Use .iloc to grab those rows
X_sample = X_test.iloc[idx]
y_true   = y_test.iloc[idx]

# Predict
y_pred_probs = model.predict(X_sample)
y_pred = np.argmax(y_pred_probs, axis=1)

# Compare
for i, row_idx in enumerate(idx):
    true_label = int(y_true.iloc[i])   # 0 or 1
    print(f"Row {row_idx}: true={true_label}, pred={int(y_pred[i])}, probs={y_pred_probs[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Row 110: true=1, pred=1, probs=[0.04091196 0.959088  ]
Row 74: true=0, pred=0, probs=[0.99093425 0.00906574]
Row 163: true=1, pred=1, probs=[4.366146e-04 9.995634e-01]
Row 97: true=1, pred=1, probs=[3.3880243e-04 9.9966121e-01]
Row 126: true=0, pred=0, probs=[0.98494995 0.01505006]


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [32]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=150, batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    shuffle=True, verbose=1
)

Epoch 1/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 0.0188 - val_accuracy: 0.9740 - val_loss: 0.1004
Epoch 2/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0162 - val_accuracy: 0.9805 - val_loss: 0.0935
Epoch 3/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0146 - val_accuracy: 0.9805 - val_loss: 0.0890
Epoch 4/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0135 - val_accuracy: 0.9805 - val_loss: 0.0874
Epoch 5/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0129 - val_accuracy: 0.9805 - val_loss: 0.0859
Epoch 6/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0121 - val_accuracy: 0.9805 - val_loss: 0.0868
Epoch 7/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0114 - val_accuracy: 0.9805 - val_loss: 0.0868
Epoch 8/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0110 - val_accuracy: 0.9805 - v

In [33]:
import numpy as np
np.random.seed(0)  # optional, for reproducibility

# Pick 5 random row positions
idx = np.random.choice(X_test.shape[0], 5, replace=False)

# Use .iloc to grab those rows
X_sample = X_test.iloc[idx]
y_true   = y_test.iloc[idx]

# Predict
y_pred_probs = model.predict(X_sample)
y_pred = np.argmax(y_pred_probs, axis=1)

# Compare
for i, row_idx in enumerate(idx):
    true_label = int(y_true.iloc[i])   # 0 or 1
    print(f"Row {row_idx}: true={true_label}, pred={int(y_pred[i])}, probs={y_pred_probs[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Row 110: true=1, pred=1, probs=[0.00578187 0.9942181 ]
Row 74: true=0, pred=0, probs=[0.99839574 0.00160422]
Row 163: true=1, pred=1, probs=[5.780877e-05 9.999422e-01]
Row 97: true=1, pred=1, probs=[1.6352984e-05 9.9998367e-01]
Row 126: true=0, pred=0, probs=[0.9981185  0.00188155]


**Which approach(es) did you find helpful to improve your model performance?**

To explore potential improvements, I retrained the same architecture but allowed up to 150 epochs, with an EarlyStopping callback (patience = 5, monitor = validation loss).
However, Test accuracy remained essentially the same as the baseline (≈98–99%).
Model predictions became slightly more confident (probabilities closer to 0 or 1), but the predicted labels did not change.

Final Decision: Retain the simpler model since it already meets performance requirements.